## HERE Public Transit API

There are two main steps in this:
1. We request list of public transport stations reachable in given time from HERE https://developer.here.com/documentation/transit/dev_guide/topics/resource-isochrone-search.html, it returns list of stations with coordinates and minutes spent to reach it

2. We then request pedestrian isohrones from HERE https://developer.here.com/documentation/routing/dev_guide/topics/resource-calculate-isoline.html for the rest of seconds. 

For example, if we want overall time of trip being 15 minutes and it took us 10 minutes to reach certain station, we then have to draw a pedestrian isohrone for 5 minutes.

In [50]:
import pandas as pd
import folium
import requests
import re

### importing data

In [51]:
# reading api keys
api_keys = pd.read_excel('../api_keys.xlsx')
api_keys.set_index('key_name', inplace=True)

# link for mapbox map as a underlay for folium
map_url = api_keys.loc['mapbox_map']['key']

# HERE Rest api key
here_api_key = api_keys.loc['here_rest_api']['key']

In [52]:
# center point coords
center_lat, center_lon = 51.507351, -0.127660

In [53]:
# HERE API url for public transport stops
URL_pt = 'https://transit.ls.hereapi.com/v3/isochrone.json'

### request to Here API for public transport stops reachable from a point

- isohrones are requested for `15 minutes`, 
- start point is  from `center_lat, center_lon`, 
- time of departure is set as `2021-02-08T09:30:00+01:00`- 9:30 am in London.

In [54]:
#time in minutes
max_time = 15

#start time
start_time = '2021-02-08T09:30:00+01:00'

In [55]:
# start point 
start_point = f"{center_lat:.6f},{center_lon:.6f}"

# request params
params = { 
        'apiKey': here_api_key,
        'center': start_point,
        'maxDur': max_time,
        'time': start_time,
            }

# response
response = requests.get(URL_pt, params=params)
response_json = response.json()

# result in json
results = response_json['Res']['Isochrone']['IsoDest']

In [56]:
# create dataframe out of json results
stations = pd.json_normalize(results)
stations = stations.join(pd.json_normalize(stations['Stn'].explode()))
stations = stations.drop('Stn', axis=1)

# converting duration to int
stations['duration'] = pd.to_numeric(stations['duration'].str.replace("[PTM]", ""))

# rest time in seconds plus ten extra seconds to get small polygons at the ends of isohrones
stations['rest_seconds'] = (max_time*60)-(stations['duration']*60)+10
stations

,duration,y,x,name,id,rest_seconds
0,1,51.507282,-0.126594,Trafalgar Square (Stop W),420321619,850
1,1,51.508121,-0.127381,Trafalgar Square (Stop C),420321615,850
2,1,51.507160,-0.126700,Trafalgar Square (Stop X),420321620,850
3,2,51.507454,-0.129426,Trafalgar Square (Stop S),420321617,790
4,2,51.507402,-0.127720,Charing Cross,420326127,790
...,...,...,...,...,...,...
366,15,51.550344,-0.140739,Kentish Town,420326284,10
367,15,51.522320,-0.163139,London Marylebone Rail Station,418102173,10
368,15,51.503361,-0.111438,London Waterloo Rail Station,418102336,10
369,15,51.511356,-0.090398,Cannon Street,420326138,10


### pedestrian isohrones with HERE API, output for folium

works  with https://developer.here.com/documentation/routing/dev_guide/topics/resource-calculate-isoline.html

- isohrones are requested for the rest of time in seconds, taken from `stations['rest_seconds']`, 
- start points are taken from `stations`, 
- mode is pedestrian, 
- resolution is maximized

In [57]:
# HERE API url for isolines
URL_iso = 'https://isoline.route.ls.hereapi.com/routing/7.2/calculateisoline.json'

In [58]:
# returns isohrone polygons in format suitable for folium map 

def isoline_polygon_for_folium(lat, lon, time):
    #start point of isoline
    start_point = f"geo!{lat:.6f},{lon:.6f}"
    # request params
    params = { 
        'apiKey': here_api_key,
        'start': start_point,
        'range': time,
        'rangetype': 'time',
        'mode': 'fastest;pedestrian',
        'departure': 'now',
        'singlecomponent': 'true',
        'resolution': 10,
        'maxpoints': 300,
            }
    # response
    response = requests.get(URL_iso, params=params)
    response_json = response.json()
    
    try:
        # resulting polygon
        polygon = response_json['response']['isoline'][0]['component'][0]['shape']

        # creating dataframe with lon lat columns out of result
        poly_df = pd.DataFrame(polygon)
        coordinates = poly_df[0].str.split(",", expand = True)
        coordinates.columns = ['lat', 'lon']
       
        # lists of cordinates
        lat = coordinates['lat'].astype('float').to_list()
        lon = coordinates['lon'].astype('float').to_list()

        # tuple for polygon
        points = []
        for lat, lon in zip(lat, lon):
            points.append(tuple([lat, lon]))

        return points

    except Exception as e:
        print("for point", start_point)
        print("result is", response_json)
        print("which raises", e)
        return ""

In [59]:
stations['isoline_folium'] = stations.apply(lambda stations: isoline_polygon_for_folium
                                        (stations['y'],stations['x'], stations['rest_seconds']), axis=1)

stations.head()

,duration,y,x,name,id,rest_seconds,isoline_folium
0,1,51.507282,-0.126594,Trafalgar Square (Stop W),420321619,850,"[(51.5061378, -0.1367283), (51.5066528, -0.136..."
1,1,51.508121,-0.127381,Trafalgar Square (Stop C),420321615,850,"[(51.5061378, -0.1374149), (51.5066528, -0.137..."
2,1,51.507160,-0.126700,Trafalgar Square (Stop X),420321620,850,"[(51.5061378, -0.1367283), (51.5066528, -0.136..."
3,2,51.507454,-0.129426,Trafalgar Square (Stop S),420321617,790,"[(51.5066528, -0.1386166), (51.5068245, -0.138..."
4,2,51.507402,-0.127720,Charing Cross,420326127,790,"[(51.5061378, -0.1374149), (51.5066528, -0.137..."


### folium map with results

In [60]:
# folium map
m = folium.Map(location=[center_lat, center_lon], tiles = map_url, zoom_start = 12, attr = 'Mapbox')

In [61]:
#add polygons of pedestrian isohrones to map

for isoline in stations['isoline_folium'].to_list():
    folium.Polygon(locations = isoline, fill_color = 'blue', opacity = 0.2).add_to(m)

# coordinates of stops
point_lat = stations['y'].to_list()
point_lon = stations['x'].to_list()

# add stops with minutes to map
for lat, lon, duration in zip(point_lat, point_lon, stations['duration']):
    folium.CircleMarker(location=[lat, lon], radius = 4, fill_color='red', 
                        color="gray", fill_opacity = 0.5, popup = duration).add_to(m)

m

In [62]:
# to export map to html
#m.save("pt_15min_here_isolines.html")

In [63]:
# export stations list to csv
#stations.to_csv('./stations_15_min.csv')